# 1.0 Setting things up

Imports all the required libraries and packages.
Checks version numbers

In [ ]:
import sys
sys.path.append('C://Users/leopo/OneDrive - ETS/Leo Wambersie/3- Données analysées/Python/OpenIO-Canada-master/OpenIO-Canada-master/')
import Working_openIO_version_CAD
import pandas as pd
import numpy as np
import regex
import pymrio
import country_converter as coco

print(pd.__version__)
print(np.__version__)
print(regex.__version__)
print(pymrio.__version__)
print(coco.__version__)

# 2.0 Running adapted version of OpenIO

Runs OpenIO code, then calculates remaining matrices.
Requires folder_path to Supply Use Tables, and exiobase pxp folder

In [ ]:
CanIO = CanCons_OpenIO.IOTables(folder_path='/15-602-x_2018_eng/Detail level/',
                                             exiobase_folder='/IOT_2018_pxp/IOT_2018_pxp/')
CanIO.calc() #Calculates remaining matrices

(Note: The following functionality has been integrated into the CanCons_OpenIO code.)

Several of the analyses require a global Z matrix. Since the A matrix was created by connecting openIO and exiobase A matrices, obtaining an estimate of a global Z matrix requires multipling the A matrix by a new vector representing overall canadian consumption, both from industry and final demand. 

In [ ]:
#Identifying import matrices
Z_io_sum = CanIO.Z_exio.loc[:, 'CA'].sum(1) #Takes Z imports from exiobase, already converted to CAD
Y_io_sum = CanIO.link_openio_exio_final_demands.sum(axis=1) # Y imports from final demand link, which is based on SUTs
# Y Alternative is CanIO.Y.iloc[6396:].sum(axis=1)
Canada_imports = Z_io_sum + Y_io_sum

#New total output vector, which appends imports to the end of the Canadian total output vector
q = CanIO.q.sum(axis=1)
q_new = pd.concat([q, Canada_imports])

# Creating a new Z matrix
test_q = pd.DataFrame(np.diag((q_new).replace(np.inf, 0)), CanIO.A.index, CanIO.A.index)
World_A = CanIO.A
World_Z = World_A.dot(test_q) 
World_Z

# 3.0 Placing construction emissions in context
Analysis conducted in spreadsheet.

In [ ]:
# All necessary information is in D matrix
CanIO.D.T.to_excel("D_Analysis.xlsx")

# 4.0 Distribution of Canadian construction emissions

Function takes the input flows to the 'Gross fixed capital formation, Construction' final consumption category, and runs an analysis of the environmental impacts embodied in each of these inputs. These inputs are all 19 forms of buildings and infrastructure, plus 3 additional categories such as architectural services, real-estate services, and legal-services.

In [ ]:
#Defines function which performs the construbution analysis and exports to excel.
def GFCF_analysis(prov):
    #Defines the Final demand sector to be analyzed
    Prov_GFCF = (prov,'Gross fixed capital formation, Construction')
    #Diagonalize
    diag_Y_Prov_GFCF = pd.DataFrame(np.diag(CanIO.Y.loc[:, Prov_GFCF].sum(1)),                                                    index=CanIO.Y.index, columns=CanIO.Y.index)
    #New E matrix
    contribution_analysis_Prov_GFCF = CanIO.C.dot(CanIO.S).dot(CanIO.L).dot(diag_Y_Prov_GFCF)
    #Groups all origin regions
    contribution_analysis_Prov_GFCF_GROUPED = contribution_analysis_Prov_GFCF.groupby(level=1, axis = 1).sum()
    # To excel
    contribution_analysis_Prov_GFCF_GROUPED.T.to_excel("GFCF_analysis"+prov+".xlsx")

In [ ]:
#Runs the construbition analysis for each province.
str_list = ['CA-AB','CA-BC','CA-MB','CA-NB','CA-NL','CA-NS','CA-NT','CA-NU','CA-ON','CA-PE','CA-SK','CA-QC','CA-YT']
for x in range(len(str_list)):
    prov = str_list[x]
    GFCF_analysis(prov)

# 5.0 Breaking down sources of emissions within construction sectors
This code performs a contribution analysis on the inputs to each of the 4 categories of buildings/infrastructure listed below. 
Because these are inter-industry flows rather than final demand flows, the Y matrix has been replaced by a column of the synthesized Z matrix.

In [ ]:
Z_temp = World_Z.copy()

str_list = ['CA-AB','CA-BC','CA-MB','CA-NB','CA-NL','CA-NS','CA-NT','CA-NU','CA-ON','CA-PE','CA-SK','CA-QC','CA-YT']

In [ ]:
# Defines function which will perform a contribution analysis for a given category of building/infrastructure, and province. 
# Note: Currently set to group by product rather than grouped by region

def construction_sector_analysis(prov,sector):
    prov = prov
    sector = sector
    #Defines the commodity in question, province
    Prov_ResB = (prov,sector)

    #Diagonalize - here it takes a column of the Z matrix, the new constructed one
    diag_Y_Prov_ResB = pd.DataFrame(np.diag(Z_temp.loc[:, Prov_ResB]),
                                                    index=Z_temp.index, columns=Z_temp.index)
    #New E matrix
    contribution_analysis_Prov_ResB = CanIO.C.dot(CanIO.S).dot(CanIO.L).dot(diag_Y_Prov_ResB)

    #Test
    contribution_analysis_Prov_ResB.loc[('Climate change, short term','kg CO2 eq (short)')].T.sort_values(ascending=False)
    contribution_analysis_Prov_ResB

    #Groups all regions
    contribution_analysis_Prov_ResB_GROUPED = contribution_analysis_Prov_ResB.groupby(level=1, axis = 1).sum() #level = 1 for group by product, 0 for group by province

    # To excel 
    contribution_analysis_Prov_ResB_GROUPED.T.to_excel("ByProduct_"+prov+"_"+sector+".xlsx")

In [ ]:
#Runs the analysis for residential buildings, for all provinces.
sector = "Residential buildings"

for x in range(len(str_list)):
    prov = str_list[x]
    construction_sector_analysis(prov, sector)

In [ ]:
sector = "Highways, roads, streets, bridges and tunnels"

for x in range(len(str_list)):
    prov = str_list[x]
    construction_sector_analysis(prov, sector)

In [ ]:
sector = "Electric power engineering works"

for x in range(len(str_list)):
    prov = str_list[x]
    construction_sector_analysis(prov, sector)

In [ ]:
sector = "Communications networks"

for x in range(len(str_list)):
    prov = str_list[x]
    construction_sector_analysis(prov, sector)

# 6.0 Mapping flows of embodied emissions in residential building construction

Residential building construction chosen as a representative sector. For each province or ROW region, the function zeroes out the GHG contribution of all sectors in the S matrix. A contribution analysis is then run, and any decrease in embodied flows caused by the removal of emissions originating the region are deemed equivalent to that region's constribution to embodied emissions flows. Note: 'Scenario list' term used because of additional research testing sector-based scenarios.

In [ ]:
#establishing parameteres

ROW = ['AT', 'AU', 'BE', 'BG', 'BR', 'CA', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR',
       'GB', 'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU',
       'LV', 'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SE', 'SI', 'SK',
       'TR', 'TW', 'WA', 'WE', 'WF', 'WL', 'WM', 'ZA']

scenario_list = ['CA-AB','CA-BC','CA-MB','CA-NB','CA-NL','CA-NS','CA-NT','CA-NU',
                 'CA-ON','CA-PE','CA-SK','CA-QC','CA-YT','CN', 'US', ROW]
scenario_names = ['CA-AB','CA-BC','CA-MB','CA-NB','CA-NL','CA-NS','CA-NT','CA-NU',
                 'CA-ON','CA-PE','CA-SK','CA-QC','CA-YT','CN', 'US', 'ROW']
Z_temp = World_Z.copy()

prov_list = ['CA-AB','CA-BC','CA-MB','CA-NB','CA-NL','CA-NS','CA-NT','CA-NU','CA-ON','CA-PE','CA-SK','CA-QC','CA-YT']
sector = "Residential buildings"

In [ ]:
# Runs through list of alls scenarios
for a in range(len(scenario_list)):
    zeroed_sector_list = scenario_list[a] #can change
    S_zeroed = CanIO.S.copy()
    print ('running scenario '+str(a+1))
        
    #if it's a province, zero out these categories    
    if (zeroed_sector_list in prov_list):
        S_zeroed.loc[[(zeroed_sector_list, 'Carbon dioxide', 'Air'),
                      (zeroed_sector_list, 'Dinitrogen monoxide', 'Air'),
                      (zeroed_sector_list, 'Methane', 'Air')], zeroed_sector_list ] = 0
    
    #If not, zero out these others
    elif (zeroed_sector_list in ['CN', 'US', ROW]):
            S_zeroed.loc[[('CO2 - combustion - air'),
                            ('CH4 - combustion - air'),
                            ('N2O - combustion - air'),
                            ('CO2 - non combustion - Cement production - air'),
                            ('CO2 - non combustion - Lime production - air'),
                            ('CH4 - non combustion - Extraction/production of (natural) gas - air'),
                            ('CH4 - non combustion - Extraction/production of crude oil - air'),
                            ('CH4 - non combustion - Mining of antracite - air'),
                            ('CH4 - non combustion - Mining of bituminous coal - air'),
                            ('CH4 - non combustion - Mining of coking coal - air'),
                            ('CH4 - non combustion - Mining of lignite (brown coal) - air'),
                            ('CH4 - non combustion - Mining of sub-bituminous coal - air'),
                            ('CH4 - non combustion - Oil refinery - air'),
                            ('CH4 - agriculture - air'),
                            ('CO2 - agriculture - peat decay - air'),
                            ('N2O - agriculture - air'),
                            ('CH4 - waste - air'),
                            ('CO2 - waste - biogenic - air'),
                            ('CO2 - waste - fossil - air')], zeroed_sector_list] = 0
                
                
    #This function runs an analysis with the zeroed out S matrix for the given sector under investigation        
    for z in range(len(prov_list)):
        #Run classic sensitivity analysis with new S
        prov = prov_list[z]
        #Diagonalize - here it takes a column of the Z matrix, the new constructed one
        diag_Y_PSec = pd.DataFrame(np.diag(Z_temp.loc[:, (prov,sector)]),
                                                        index=Z_temp.index, columns=Z_temp.index)
        #New E matrix, calculated using the new S matrix from above
        contribution_analysis_Psec = CanIO.C.dot(S_zeroed).dot(CanIO.L).dot(diag_Y_PSec)

        #Groups all regions (and transposes)
        contribution_analysis_Psec_GROUPED = contribution_analysis_Psec.groupby(level=1, axis = 1).sum().T #level = 1 for group by product, 0 for group by province

        #Adds province lavel
        contribution_analysis_Psec_GROUPED = pd.concat([contribution_analysis_Psec_GROUPED], keys=[prov], names=['Province'])    

        # Appends each table below the previous one
        if z == 0 :
            AllProv_contribution_analysis_Psec_GROUPED = contribution_analysis_Psec_GROUPED
            print("analysis for " + prov + "- Create df")
        elif z > 0 :
            AllProv_contribution_analysis_Psec_GROUPED = AllProv_contribution_analysis_Psec_GROUPED.append(contribution_analysis_Psec_GROUPED)
            print("analysis for " + prov + "- appending")
    #Test view        
    #AllProv_contribution_analysis_Psec_GROUPED

    # To excel 
    print("to excel")
    AllProv_contribution_analysis_Psec_GROUPED.to_excel("Zeroed_Region_"+scenario_names[a]+"_"+sector+"analysis.xlsx") 

# 7.0 Comparison with embodied energy

Based on data from # 3.0. Analysis performed in spreadsheet.

# 8.0 Efficiency of construction activities - kgCO2 and TJ per dollar

Based on data from # 3.0 as well as a snapshot of the Y matrix.

In [ ]:
Y_grouped = CanIO.Y.groupby(level=[0,1],axis = 1).sum()
Y_grouped = Y_grouped.loc[:,
              [('CA-AB','Gross fixed capital formation, Construction'),
              ('CA-BC','Gross fixed capital formation, Construction'),
              ('CA-MB','Gross fixed capital formation, Construction'),
              ('CA-NB','Gross fixed capital formation, Construction'),
              ('CA-NL','Gross fixed capital formation, Construction'),
              ('CA-NS','Gross fixed capital formation, Construction'),
              ('CA-NT','Gross fixed capital formation, Construction'),
              ('CA-NU','Gross fixed capital formation, Construction'),
              ('CA-ON','Gross fixed capital formation, Construction'),
              ('CA-PE','Gross fixed capital formation, Construction'),
              ('CA-QC','Gross fixed capital formation, Construction'),
              ('CA-SK','Gross fixed capital formation, Construction'),
              ('CA-YT','Gross fixed capital formation, Construction')]
             ]
Y_grouped
Y_grouped.to_excel("Y_GFCF_detailed.xlsx")